## Oracle Database RAG with Gemini & Vertex AI

Run the RAG application code snippets in this Jupyter notebook.

Execute each code cell in sequence from top to bottom. To run a code cell, select it and click Run. When a cell completes, a number will appear in the square brackets. You can then proceed to the next cell.

**Prerequisites:**
- Python virtual environment activated with all required packages
- GCP authentication completed (`gcloud auth application-default login`)
- `.env` file configured with database credentials

**Libraries used:**
- LangChain for RAG orchestration
- Oracle AI Vector Search (OracleVS) for vector storage
- Vertex AI for embeddings and LLM (Gemini)

In [ ]:
# Import libraries and modules

import time
import os
import warnings

# Suppress tqdm warning about ipywidgets
warnings.filterwarnings('ignore', message='IProgress not found')

from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

from langchain_google_vertexai import (
    ChatVertexAI,
    VertexAIEmbeddings,
    VertexAI,
)

print('✓ Successfully imported libraries and modules')

## Step 1: Define metadata wrapper function

This function formats and adds metadata to chunks for the Oracle Vector Store.

In [ ]:
# Function to format and add metadata to Oracle 23ai Vector Store
from langchain_core.documents import Document

def chunks_to_docs_wrapper(row: dict) -> Document:
    """
    Converts text into a Document object suitable for ingestion into Oracle Vector Store.
    """
    metadata = {'id': row['id'], 'link': row['link']}
    return Document(page_content=row['text'], metadata=metadata)

print("Successfully defined metadata wrapper")

## Step 2: Connect to Oracle Database 23ai

Update with your credentials:
- **Username**
- **Password** 
- **Connection String** (from tnsnames.ora)
- **Wallet Password**

In [ ]:
import oracledb
from dotenv import load_dotenv

# Load environment variables from .env file (specify explicit path for notebooks)
env_path = os.path.join(os.getcwd(), '.env')
# Use override=True to ensure .env values take precedence over existing env vars
load_dotenv(dotenv_path=env_path, override=True)

# Get database credentials from environment variables
un = os.getenv("DB_USERNAME")
pw = os.getenv("DB_PASSWORD")
dsn = os.getenv("DB_DSN")
wallet_path = os.getenv("DB_WALLET_DIR")
wpwd = os.getenv("DB_WALLET_PASSWORD", "")

# Debug: Show what we loaded (mask password)
print(f"Loaded from .env:")
print(f"  DB_USERNAME: {un}")
print(f"  DB_DSN: {dsn}")
print(f"  DB_WALLET_DIR: {wallet_path}")
print(f"  DB_WALLET_PASSWORD: {'*' * len(wpwd) if wpwd else '(empty)'}")
print()

connection = oracledb.connect(
    config_dir=wallet_path,
    user=un, 
    password=pw, 
    dsn=dsn,
    wallet_location=wallet_path,
    wallet_password=wpwd
)

print("Successfully connected to Oracle Database 26ai")

## Step 3: Load PDF Document

Load the PDF document and display basic information about it.

In [ ]:
import requests
from io import BytesIO
from PyPDF2 import PdfReader

# Download PDF from Oracle documentation
pdf_url = 'https://docs.oracle.com/en/database/oracle/oracle-database/26/nfcoa/oracle-ai-database-26ai-new-features-guide.pdf'
print(f"Downloading PDF from: {pdf_url}")

response = requests.get(pdf_url)
response.raise_for_status()  # Raise error if download fails

# Load PDF from downloaded bytes
pdf = PdfReader(BytesIO(response.content))
print(f"✓ Successfully downloaded PDF")
print(f"The number of pages in this document is {len(pdf.pages)}")
print("\n--- First Page Preview ---")
print(pdf.pages[0].extract_text())

## Step 4: Transform PDF to Text

Extract text from all pages of the PDF document.

In [ ]:
if pdf is not None:
    print("Transforming the PDF document to text...")
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    print(f"Successfully transformed {len(pdf.pages)} pages to text")
    print(f"Total text length: {len(text)} characters")

## Step 5: Split Text into Chunks

Chunk size: 800 characters with 100 character overlap.

**Note:** Chunk sizes vary depending on document type.

In [ ]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
)

chunks = text_splitter.split_text(text)
print(f"Created {len(chunks)} chunks")
print("\n--- First Chunk Preview ---")
print(chunks[0])

## Step 6: Create Documents with Metadata

Wrap each chunk with metadata (id and link) for storage in the vector database.

In [ ]:
docs = [
    chunks_to_docs_wrapper({
        'id': f'{page_num}', 
        'link': f'Page {page_num}', 
        'text': text
    }) 
    for page_num, text in enumerate(chunks)
]

print(f"Created {len(docs)} documents with metadata")

## Step 7: Initialize Vertex AI

Configure your Google Cloud project and region for Vertex AI services.

## Step 6a: Authenticate with GCP (First Time Setup)

**Run this ONCE on your GCP VM to set up authentication:**

```bash
# In the VM terminal (not in notebook):
gcloud auth application-default login --no-launch-browser
```

Follow the URL, copy the authorization code, and paste it back.

**OR** if you have a service account key file:
```bash
export GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/service-account-key.json"
```

⚠️ **After authentication, restart the kernel and re-run from the top.**

In [ ]:
import vertexai

PROJECT_ID = "adb-pm-prod"  # Update with your GCP Project ID
REGION = "us-central1"             # Update with your region (us-central1 has most models)

vertexai.init(project=PROJECT_ID, location=REGION)
print(f"Initialized Vertex AI for project: {PROJECT_ID} in region: {REGION}")

## Step 8: Embed and Store Vectors in Oracle 23ai

Using **VertexAIEmbeddings** model and **DOT_PRODUCT** distance strategy for similarity search.

In [ ]:
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy

embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

s1time = time.time()
print(f"Vectorizing and inserting {len(docs)} chunks into Oracle Database 23ai...")
print("Processing in smaller batches to avoid token limits...")

# Process in batches of 50 (to stay under 20k token limit)
batch_size = 50
knowledge_base = None

for i in range(0, len(docs), batch_size):
    batch = docs[i:i + batch_size]
    batch_num = (i // batch_size) + 1
    total_batches = (len(docs) + batch_size - 1) // batch_size
    print(f"  Processing batch {batch_num}/{total_batches} ({len(batch)} chunks)...")
    
    if knowledge_base is None:
        # Create the vector store with first batch
        knowledge_base = OracleVS.from_documents(
            batch,
            embeddings,
            client=connection,
            table_name="RAG_TAB",
            distance_strategy=DistanceStrategy.DOT_PRODUCT
        )
    else:
        # Add remaining batches to existing store
        knowledge_base.add_documents(batch)

s2time = time.time()
print(f"✓ Vectorizing and inserting chunks duration: {round(s2time - s1time, 1)} sec.")
print(f"✓ Successfully stored {len(docs)} chunks in Oracle Database 23ai")

## Step 9: Verify Data in Oracle Database

Query the RAG_TAB table to confirm vectors were inserted successfully.

In [ ]:
table_name = "RAG_TAB"

with connection.cursor() as cursor:
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    rows = cursor.fetchall()
    
    print(f"Total rows in {table_name}: {len(rows)}")
    print("\n--- Sample Rows (first 3) ---")
    for row in rows[:3]:
        print(row)

## Step 10: Define User Question

Ask a question about the document content.

In [ ]:
user_question = 'Tell me more about JSON Relational Duality'
print(f"The prompt to the LLM will be: {user_question}")

## Step 11: Perform Similarity Search

Test the vector similarity search to find relevant chunks.

In [ ]:
if user_question:
    s3time = time.time()
    result_chunks = knowledge_base.similarity_search(user_question, k=5)
    s4time = time.time()
    
    print(f"✓ Search duration: {round(s4time - s3time, 1)} sec.")
    print(f"\nFound {len(result_chunks)} relevant chunks:")
    for i, chunk in enumerate(result_chunks, 1):
        print(f"\nChunk {i}: {chunk.page_content[:200]}...")

## Step 12: Configure Gemini LLM

Set up Vertex AI's **Gemini 2.0 Flash** model for generating responses.

In [ ]:
llm = ChatVertexAI(
    model_name="gemini-2.5-flash",
    max_output_tokens=8192,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    verbose=True
)

print("✓ Configured Gemini 2.5 Flash model")

## Step 13: Build RAG Prompt Template

Create the prompt template and retriever for the RAG pipeline.

In [ ]:
template = """Answer the question based only on the following context:
            {context} Question: {question} """
prompt = PromptTemplate.from_template(template)
retriever = knowledge_base.as_retriever(search_kwargs={"k": 10})
print("The template is:", template)
print(retriever)

## Step 14: Execute RAG Chain

Invoke the complete RAG pipeline to generate the final response.

The chain:
1. Retrieves relevant context from Oracle Vector DB
2. Constructs prompt with question + context
3. Sends to Gemini LLM for response generation

In [ ]:
s5time = time.time()
print("Sending prompt and RAG context to Gemini LLM...")
print(f"Question: {user_question}\n")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke(user_question)

s6time = time.time()

print("=" * 80)
print("RESPONSE:")
print("=" * 80)
print(response)
print("=" * 80)
print(f"\n✓ LLM response duration: {round(s6time - s5time, 1)} sec.")

## 🎉 Congratulations!

You've successfully built a complete RAG application with:
- **Oracle Database 23ai** for vector storage
- **Vertex AI Embeddings** for vectorization  
- **Gemini 2.0 Flash** for response generation
- **LangChain 1.x** for orchestration

In [ ]:
print("=" * 80)
print("🎉 RAG Application Complete!")
print("=" * 80)
print("\n✓ PDF loaded and processed")
print("✓ Text chunked and vectorized") 
print("✓ Vectors stored in Oracle Database 23ai")
print("✓ Similarity search working")
print("✓ Gemini LLM integration successful")
print("\nYou've completed the RAG application lab!")
print("=" * 80)

## Step 15: Create PL/SQL Package for REST API

Now we'll expose the vector search functionality as a REST/OpenAPI endpoint that can be consumed by Dialogflow agents.

This creates a PL/SQL package with two main functions:
- `search_similar_chunks()` - Performs vector similarity search
- `health_check()` - Health check endpoint

In [ ]:
# Reconnect to database if connection timed out
try:
    with connection.cursor() as cursor:
        cursor.execute("SELECT 1 FROM DUAL")
    print("✓ Database connection is active")
except:
    print("⚠️ Connection lost, reconnecting...")
    
    # Redefine connection parameters
    un = "ADMIN"
    pw = "password"
    dsn = 'paulparkdb_high'
    wpwd = "YourWalletPassword123#"
    wallet_path = os.path.expanduser('~/wallet')
    
    connection = oracledb.connect(
        config_dir=wallet_path,
        user=un, 
        password=pw, 
        dsn=dsn,
        wallet_location=wallet_path,
        wallet_password=wpwd
    )
    print("✓ Reconnected to Oracle Database 23ai")

In [ ]:
# Create Package Specification
sql_spec = """
CREATE OR REPLACE PACKAGE rag_search_api AS
    -- Search for similar document chunks using vector similarity
    FUNCTION search_similar_chunks(
        p_question IN VARCHAR2,
        p_embedding_json IN CLOB,
        p_top_k IN NUMBER DEFAULT 10
    ) RETURN CLOB;
    
    -- Health check endpoint
    FUNCTION health_check RETURN CLOB;
END rag_search_api;
"""

with connection.cursor() as cursor:
    cursor.execute(sql_spec)
    print("✓ Package specification created successfully")


In [ ]:
# Create Package Body - Corrected TO_VECTOR syntax
sql_body = """
CREATE OR REPLACE PACKAGE BODY rag_search_api AS
    
    FUNCTION search_similar_chunks(
        p_question IN VARCHAR2,
        p_embedding_json IN CLOB,
        p_top_k IN NUMBER DEFAULT 10
    ) RETURN CLOB IS
        l_result JSON_OBJECT_T;
        l_chunks JSON_ARRAY_T;
        l_chunk JSON_OBJECT_T;
        l_embedding VECTOR;
        l_id VARCHAR2(4000);
        l_text CLOB;
        l_metadata CLOB;
        l_distance NUMBER;
    BEGIN
        -- Initialize JSON response
        l_result := JSON_OBJECT_T();
        l_chunks := JSON_ARRAY_T();
        
        -- Convert JSON array to VECTOR type (correct syntax: no asterisk)
        l_embedding := TO_VECTOR(p_embedding_json);
        
        -- Perform vector similarity search
        FOR rec IN (
            SELECT 
                id,
                text,
                cmetadata,
                VECTOR_DISTANCE(embedding, l_embedding, DOT_PRODUCT) as distance
            FROM rag_tab
            ORDER BY VECTOR_DISTANCE(embedding, l_embedding, DOT_PRODUCT)
            FETCH FIRST p_top_k ROWS ONLY
        ) LOOP
            l_id := rec.id;
            l_text := rec.text;
            l_metadata := rec.cmetadata;
            l_distance := rec.distance;
            
            l_chunk := JSON_OBJECT_T();
            l_chunk.put('id', l_id);
            l_chunk.put('text', l_text);
            l_chunk.put('metadata', l_metadata);
            l_chunk.put('distance', l_distance);
            l_chunks.append(l_chunk);
        END LOOP;
        
        -- Build final response
        l_result.put('question', p_question);
        l_result.put('chunks', l_chunks);
        l_result.put('count', l_chunks.get_size());
        
        RETURN l_result.to_clob();
        
    EXCEPTION
        WHEN OTHERS THEN
            l_result := JSON_OBJECT_T();
            l_result.put('error', SQLERRM);
            RETURN l_result.to_clob();
    END search_similar_chunks;
    
    FUNCTION health_check RETURN CLOB IS
        l_result JSON_OBJECT_T;
    BEGIN
        l_result := JSON_OBJECT_T();
        l_result.put('status', 'healthy');
        l_result.put('timestamp', TO_CHAR(SYSTIMESTAMP, 'YYYY-MM-DD"T"HH24:MI:SS.FF3"Z"'));
        l_result.put('service', 'RAG Vector Search API');
        RETURN l_result.to_clob();
    END health_check;
    
END rag_search_api;
"""

with connection.cursor() as cursor:
    cursor.execute(sql_body)
    print("✓ Package body created successfully")
    
# Check for any compilation errors
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT COUNT(*) FROM USER_ERRORS 
        WHERE name = 'RAG_SEARCH_API' AND type = 'PACKAGE BODY'
    """)
    error_count = cursor.fetchone()[0]
    
    if error_count > 0:
        cursor.execute("""
            SELECT line, position, text 
            FROM USER_ERRORS 
            WHERE name = 'RAG_SEARCH_API' AND type = 'PACKAGE BODY'
            ORDER BY sequence
        """)
        print("⚠️ Compilation errors found:")
        for line, pos, text in cursor.fetchall():
            print(f"  Line {line}, Pos {pos}: {text}")
    else:
        print("✓ No compilation errors")


## Step 16: Configure ORDS REST Endpoints

Configure Oracle REST Data Services (ORDS) to expose the PL/SQL functions as REST endpoints.

In [ ]:
# Configure ORDS REST endpoints
# First, try to delete existing module if it exists
try:
    with connection.cursor() as cursor:
        cursor.execute("BEGIN ORDS.DELETE_MODULE(p_module_name => 'rag_api'); COMMIT; END;")
        print("✓ Deleted existing module")
except:
    print("ℹ No existing module to delete")

# Now create the module and endpoints
sql_ords = """
BEGIN
    -- Define the REST module (module name cannot contain dots)
    ORDS.DEFINE_MODULE(
        p_module_name => 'rag_api',
        p_base_path => '/rag/api/v1/',
        p_items_per_page => 0,
        p_status => 'PUBLISHED',
        p_comments => 'RAG Vector Search REST API'
    );
    
    -- Define POST endpoint for vector search
    ORDS.DEFINE_TEMPLATE(
        p_module_name => 'rag_api',
        p_pattern => 'search',
        p_priority => 0,
        p_etag_type => 'HASH',
        p_etag_query => NULL,
        p_comments => 'Search for similar document chunks'
    );
    
    ORDS.DEFINE_HANDLER(
        p_module_name => 'rag_api',
        p_pattern => 'search',
        p_method => 'POST',
        p_source_type => 'plsql/block',
        p_source => 'BEGIN
            :result := rag_search_api.search_similar_chunks(
                p_question => :question,
                p_embedding_json => :embedding,
                p_top_k => NVL(:top_k, 10)
            );
        END;',
        p_items_per_page => 0,
        p_comments => 'Search similar chunks by embedding vector'
    );
    
    -- Define GET endpoint for health check
    ORDS.DEFINE_TEMPLATE(
        p_module_name => 'rag_api',
        p_pattern => 'health',
        p_priority => 0,
        p_etag_type => 'HASH',
        p_etag_query => NULL,
        p_comments => 'Health check endpoint'
    );
    
    ORDS.DEFINE_HANDLER(
        p_module_name => 'rag_api',
        p_pattern => 'health',
        p_method => 'GET',
        p_source_type => 'plsql/block',
        p_source => 'BEGIN
            :result := rag_search_api.health_check();
        END;',
        p_items_per_page => 0,
        p_comments => 'Check API health status'
    );
    
    COMMIT;
END;
"""

with connection.cursor() as cursor:
    cursor.execute(sql_ords)
    print("✓ ORDS endpoints configured successfully")
    print(f"✓ Endpoints available at: <YOUR_ORDS_URL>/ords/{un.lower()}/rag/api/v1/")

## Step 17: Test the REST Endpoints

Test the health check endpoint and the vector search endpoint.

## Step 16b: Test PL/SQL Function Directly

Before testing via ORDS, let's verify the PL/SQL function works by calling it directly from Python.

In [ ]:
# Check compilation errors
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT line, position, text 
        FROM USER_ERRORS 
        WHERE name = 'RAG_SEARCH_API' 
        AND type = 'PACKAGE BODY'
        ORDER BY sequence
    """)
    errors = cursor.fetchall()
    
    if errors:
        print("❌ Package Body Compilation Errors:")
        print("=" * 80)
        for line, pos, text in errors:
            print(f"Line {line}, Position {pos}: {text}")
    else:
        print("✅ No compilation errors found")

In [ ]:
# Check RAG_TAB table structure
with connection.cursor() as cursor:
    cursor.execute("""
        SELECT column_name, data_type, data_length
        FROM USER_TAB_COLUMNS 
        WHERE table_name = 'RAG_TAB'
        ORDER BY column_id
    """)
    columns = cursor.fetchall()
    
    print("RAG_TAB Table Structure:")
    print("=" * 80)
    for col_name, data_type, data_len in columns:
        print(f"{col_name:20s} {data_type:20s} {data_len if data_len else 'N/A'}")
    print("=" * 80)

In [ ]:
import json

# Test 1: Health Check
print("Testing Health Check Function...")
print("-" * 50)
with connection.cursor() as cursor:
    # Use callfunc() for functions that return values
    # callfunc(name, return_type, parameters)
    health_result = cursor.callfunc('rag_search_api.health_check', oracledb.CLOB)
    print(f"Health Check Result:\n{health_result}")
    health_json = json.loads(health_result)
    print(f"\nParsed JSON: {json.dumps(health_json, indent=2)}")

print("\n")

# Test 2: Search Function with Dummy Embedding
print("Testing Search Function...")
print("-" * 50)

# Create a dummy 768-dimensional embedding (all zeros for testing)
dummy_embedding = [0.0] * 768
embedding_json = json.dumps(dummy_embedding)

with connection.cursor() as cursor:
    # callfunc(name, return_type, [param1, param2, param3])
    search_result = cursor.callfunc(
        'rag_search_api.search_similar_chunks',
        oracledb.CLOB,
        ["Test question", embedding_json, 3]
    )
    print(f"Search Result:\n{search_result}")
    search_json = json.loads(search_result)
    print(f"\nParsed JSON: {json.dumps(search_json, indent=2)}")
    print(f"\nFound {search_json.get('count', 0)} chunks")

print("\n✅ PL/SQL functions work correctly!")
print("\n⚠️ If you see results above, the PL/SQL code is fine.")
print("The HTTP 555 error is an ORDS configuration issue, not a PL/SQL problem.")
print("\nNext steps:")
print("1. Check ORDS console: https://gy5twnrd.adb.us-ashburn-1.oraclecloudapps.com/ords/sql-developer")
print("2. Navigate to: REST > Modules > rag_api")
print("3. Verify handlers are PUBLISHED and patterns match")
print("4. Try accessing: https://gy5twnrd.adb.us-ashburn-1.oraclecloudapps.com/ords/admin/rag_api/health")
print("   (Note: might need to adjust schema name or path)")

In [ ]:
import requests
import json
from vertexai.language_models import TextEmbeddingModel

# Your ORDS Base URL (ends with /ords/)
ORDS_BASE_URL = "https://gy5twnrd.adb.us-ashburn-1.oraclecloudapps.com/ords/"

# Username from Step 2
SCHEMA_NAME = un.lower()  # Should be 'admin'

# Construct full URL: base + schema + module path
BASE_PATH = f"{ORDS_BASE_URL}{SCHEMA_NAME}/rag/api/v1/"

print(f"Schema: {SCHEMA_NAME}")
print(f"Base Path: {BASE_PATH}")
print("=" * 80)

# Test health check endpoint
print("Testing Health Check Endpoint...")
health_url = f"{BASE_PATH}health"
print(f"URL: {health_url}")

try:
    response = requests.get(health_url, timeout=10)
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        try:
            result = response.json()
            print(f"✅ Response: {json.dumps(result, indent=2)}")
        except:
            print(f"Response text: {response.text[:500]}")
    else:
        print(f"❌ Error response: {response.text[:500]}")
except Exception as e:
    print(f"❌ Connection Error: {e}")

# Test vector search endpoint
print("\n" + "="*80)
print("Testing Vector Search Endpoint...")

try:
    # Generate embedding for a test question
    test_question = "What is Oracle Database 23ai?"
    print(f"Generating embedding for: {test_question}")
    embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
    embedding_response = embedding_model.get_embeddings([test_question])
    embedding_vector = list(embedding_response[0].values)  # Convert to list
    
    # Call the ORDS endpoint
    search_url = f"{BASE_PATH}search"
    print(f"URL: {search_url}")
    payload = {
        "question": test_question,
        "embedding": embedding_vector,
        "top_k": 5
    }
    
    response = requests.post(search_url, json=payload, timeout=30)
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        try:
            result = response.json()
            print(f"✅ Found {result.get('count', 0)} matching chunks")
            print(f"Response: {json.dumps(result, indent=2)[:1000]}...")  # Truncate for readability
            print("\n" + "="*80)
            print("✅ REST API endpoints are working!")
        except:
            print(f"Response text: {response.text[:500]}")
    else:
        print(f"❌ Error response: {response.text[:500]}")
        
except Exception as e:
    print(f"❌ Error: {e}")

## Step 18: OpenAPI Specification for Dialogflow Integration

This OpenAPI specification can be used to integrate the REST API with Google Cloud Dialogflow agents.

Save this as `rag-api-openapi.yaml` and use it when configuring a Dialogflow CX agent tool.

In [ ]:
openapi: 3.0.0
info:
  title: Oracle RAG Vector Search API
  description: REST API for performing vector similarity search on Oracle Database 23ai with RAG functionality
  version: 1.0.0
  contact:
    name: API Support
    email: support@example.com

servers:
  - url: https://your-ords-url.oraclecloudapps.com/ords/admin/rag/api/v1
    description: Oracle Autonomous Database ORDS endpoint

paths:
  /search:
    post:
      summary: Search for similar document chunks
      description: Performs vector similarity search using document embeddings
      operationId: searchSimilarChunks
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              required:
                - question
                - embedding
              properties:
                question:
                  type: string
                  description: The user's question text
                  example: "What is Oracle Database 23ai?"
                embedding:
                  type: array
                  items:
                    type: number
                    format: float
                  description: Vector embedding of the question (768 dimensions for text-embedding-004)
                  example: [0.123, -0.456, 0.789, ...]
                top_k:
                  type: integer
                  default: 10
                  description: Number of similar chunks to return
                  example: 5
      responses:
        '200':
          description: Successful search results
          content:
            application/json:
              schema:
                type: object
                properties:
                  question:
                    type: string
                    description: The original question
                  chunks:
                    type: array
                    items:
                      type: object
                      properties:
                        id:
                          type: string
                          description: Chunk ID
                        text:
                          type: string
                          description: Document chunk text
                        metadata:
                          type: object
                          description: Document metadata
                        distance:
                          type: number
                          description: Similarity distance score
                  count:
                    type: integer
                    description: Number of chunks returned
        '400':
          description: Bad request
        '500':
          description: Internal server error

  /health:
    get:
      summary: Health check endpoint
      description: Check if the API is operational
      operationId: healthCheck
      responses:
        '200':
          description: API is healthy
          content:
            application/json:
              schema:
                type: object
                properties:
                  status:
                    type: string
                    example: "healthy"
                  timestamp:
                    type: string
                    format: date-time
                  service:
                    type: string
                    example: "RAG Vector Search API"

## Step 19: Dialogflow Agent Integration Instructions

To integrate this REST API with a Dialogflow CX agent:

### Option 1: Two-Step Approach (Recommended)
1. **Create an embedding generation service** that calls Vertex AI to convert text questions to embeddings
2. **Configure Dialogflow tool** to call your embedding service first, then call the ORDS endpoint

### Option 2: Enhanced PL/SQL with DBMS_CLOUD
Create a wrapper function that:
1. Takes a text question as input
2. Uses DBMS_CLOUD.SEND_REQUEST to call Vertex AI Embeddings API
3. Passes the embedding to the search function
4. Returns results

### Dialogflow Agent Configuration
1. Go to Dialogflow CX console
2. Create or open your agent
3. Navigate to **Tools** section
4. Click **+ Create**
5. Upload the OpenAPI YAML specification from Step 18
6. Configure authentication (OAuth 2.0 or API Key for ORDS)
7. In your agent's instructions, reference the tool:
   ```
   When the user asks about Oracle Database, use ${TOOL: searchSimilarChunks} 
   to find relevant information from the documentation.
   ```

### Example Agent Instruction
```
You are an Oracle Database expert assistant. When users ask questions about 
Oracle Database features, capabilities, or documentation:

1. Generate an embedding for the user's question using Vertex AI
2. Use ${TOOL: searchSimilarChunks} with the embedding to retrieve relevant 
   documentation chunks
3. Synthesize the information from the retrieved chunks to provide a 
   comprehensive answer
4. Always cite the source documents in your response
```

### Testing the Integration
After configuration, test with questions like:
- "What are the vector database features in Oracle 23ai?"
- "How do I use SELECT AI in Oracle Database?"
- "What is JSON Relational Duality in Oracle?"

## Summary of Deployment Options

You now have **four different ways** to deploy and use this RAG system:

### 1. **Jupyter Notebook** (This notebook)
- **Use case**: Development, experimentation, data science workflows
- **Pros**: Interactive, easy to modify, great for prototyping
- **Cons**: Not suitable for production, requires Jupyter environment

### 2. **Java Spring Boot REST API** 
- **Location**: `oracle-ai-database-gcp-vertex-ai/rag-ai-agent-java/`
- **Use case**: Enterprise microservices, Spring ecosystem integration
- **Pros**: Type-safe, robust error handling, Spring Boot ecosystem
- **Cons**: Requires JVM, larger memory footprint

### 3. **Python Flask REST API**
- **Location**: `oracle-ai-database-gcp-vertex-ai/rag-ai-agent-python/`
- **Use case**: Lightweight REST API, ML/AI workloads
- **Pros**: Lightweight, Python ecosystem, easy to deploy
- **Cons**: Less type-safe than Java

### 4. **PL/SQL ORDS REST API** (Steps 15-19)
- **Use case**: Database-native deployment, minimal infrastructure
- **Pros**: No external compute needed, runs in Oracle Database, direct ORDS exposure
- **Cons**: Requires embedding generation middleware or DBMS_CLOUD wrapper

### Choosing the Right Approach

| Requirement | Best Option |
|------------|-------------|
| Minimal infrastructure | PL/SQL ORDS |
| Existing Spring Boot apps | Java Spring Boot |
| Python/ML ecosystem | Python Flask |
| Development & testing | Jupyter Notebook |
| Dialogflow integration | Any REST API option |
| Lowest latency | PL/SQL ORDS (in-database) |
| Most flexibility | Java or Python |

**Next Steps:**
- Deploy the option that best fits your architecture
- Configure Dialogflow CX agent with the OpenAPI specification
- Test end-to-end with real user questions
- Monitor performance and adjust `top_k` parameter as needed

In [ ]:
# Test PL/SQL function directly (bypass ORDS for now)
import json

# Test health check function
print("Testing PL/SQL health_check function...")
with connection.cursor() as cursor:
    result_var = cursor.var(oracledb.CLOB)
    cursor.callproc('rag_search_api.health_check', [result_var])
    health_result = result_var.getvalue()
    print(f"Health Check Result: {health_result}")
    print()

# Test search function with a dummy embedding
print("Testing PL/SQL search_similar_chunks function...")
with connection.cursor() as cursor:
    # Create a small test embedding (768 zeros - won't match well but tests the function)
    test_embedding = [0.0] * 768
    test_embedding_json = json.dumps(test_embedding)
    
    result_var = cursor.var(oracledb.CLOB)
    cursor.callproc('rag_search_api.search_similar_chunks', 
                    ['test question', test_embedding_json, 3, result_var])
    search_result = result_var.getvalue()
    
    print(f"Search Result: {search_result[:500]}...")
    print()
    print("✅ PL/SQL functions are working!")
    print()
    print("⚠️  ORDS Issue: Status 555 means the endpoint isn't properly registered.")
    print("Next steps:")
    print("1. Check ORDS console - the handlers should be visible")
    print("2. Try accessing: https://gy5twnrd.adb.us-ashburn-1.oraclecloudapps.com/ords/sql-developer")
    print("3. Navigate to REST > Modules > rag_api to verify configuration")